This notebook is a hands-on introduction to prompting LLMs. We will be will using OpenAI's GTP 3.5 Turbo. The skills that you learn here are transferrable to any LLM. 

In practice, we have found that GPT 4 gives slightly better performance at 10x the cost. Your employer may prefer to stick with GPT 3.5 Turbo. Especially when making 1000's of requests per day.

The prompts in this notebook are "single-turn instructions". We will be issuing single instructions to the LLM, rather than having conversations. Although `gpt-3.5-turbo` is "optimised for conversation", it works for single-turn instructions as well. 

### Learning Outcomes:
- Calling an OpenAI "Chat Completion" LLM from Python.
- Iteratively refining prompts to get better outcomes. Prompts usually improve as we get more specific.
- Techniques for improving prompts.
- Using the "temparature" setting on an LLM.
- An introduction to _Chain-Of-Thought_ prompting and how it generalises. It can also be called _giving the llm time to think_ or _specifying the steps to solve a problem_.
- Inducing and mitigating hallucinations in LLMs
- _EmotionPrompt_, enhancing LLM output with emotional stimuli.


# Part 1: A scenario with iterative prompt improvement

#### Our first API call and LLM prompt
We'll be asking the LLM give us advice on getting promoted. We will iteratively improve our prompt to get better and more specific answers. 

The API calls might look a bit repetitive in the first part. Because I would like to demonstrate the `openai` python package before reducing boilerplate code.

In [1]:
#Step 0: Import the openai package and set the API key. 
#         I have my API key stored in an environment variable for this demo.
#         In prod, you might prefer to use a secret store.
import openai
import os

openai.api_key  = os.getenv('MY_API_KEY')

In [2]:
#Step 1: Write the prompt.
our_prompt = """
How can I get promoted in my job?
"""

In [3]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)

In [37]:
#Step 3: Print the response
print(response.choices[0].message.content)

As an AI language model, I don't have any information about your current job or industry. However, here are some general tips that may help you get promoted:

1. Set clear goals: Identify what you want to achieve in your current role and what you need to do to get promoted. Set specific, measurable, achievable, relevant, and time-bound (SMART) goals.

2. Improve your skills: Identify the skills and knowledge required for the next level and work on developing them. Attend training programs, take online courses, and seek feedback from your supervisor.

3. Build relationships: Build positive relationships with your colleagues, supervisors, and other stakeholders. Be a team player, collaborate, and communicate effectively.

4. Take initiative: Take on additional responsibilities, volunteer for projects, and demonstrate your willingness to go above and beyond your job description.

5. Deliver results: Focus on delivering high-quality work, meeting deadlines, and exceeding expectations. Keep

### Are you happy with that answer?
That was a bit of a dry and generic response. I also don't like being told to _"be patient"_ by a machine. Let's try again.

In [38]:
#Step 1: Improve the prompt
our_prompt = """
Suggest five ideas that I can use to get promoted in my job.
"""

In [39]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)

In [40]:
#Step 3: Print the response
print(response.choices[0].message.content)

1. Take on additional responsibilities: Show your boss that you are capable of handling more than your current workload. Volunteer for projects or tasks that are outside of your job description.

2. Improve your skills: Take courses or attend workshops to improve your skills and knowledge. This will make you more valuable to your employer and increase your chances of getting promoted.

3. Network: Build relationships with colleagues and managers in other departments. This will help you gain visibility and make it easier for you to be considered for promotions.

4. Be proactive: Don't wait for opportunities to come to you. Look for ways to improve processes or solve problems in your department. This will show your boss that you are proactive and can think outside the box.

5. Communicate your goals: Let your boss know that you are interested in advancing in the company. Ask for feedback on what you can do to improve your chances of getting promoted. This will show that you are committed

### Are you happy with that answer?

It's a little bit better. The generic warning is gone and it's no longer telling me to _"be patient"_. However, I want the advice to specific to me. I'll give it some more details.

In [1]:
#Step 1: Make the prompt more specific to my situation.
our_prompt = """
I am currently working as a Data Analyst. 
Suggest five ideas on how I can get promoted in my current role.
Include ideas that are specific to data analysts and the analytics industry.

You are always very encouraging.
"""

In [3]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)

In [4]:
#Step 3: Print the response
print(response.choices[0].message.content)

1. Develop new skills: As a data analyst, it is important to stay up-to-date with the latest tools and technologies in the industry. Consider taking courses or attending workshops to learn new skills such as machine learning, data visualization, or programming languages like Python or R.

2. Take on more responsibility: Look for opportunities to take on additional responsibilities within your current role. This could include leading projects, mentoring junior analysts, or taking ownership of specific data sets.

3. Build relationships: Networking is key in any industry, and the analytics field is no exception. Attend industry events, join professional organizations, and connect with colleagues and leaders in your company to build relationships and expand your network.

4. Demonstrate value: Show your value to the company by consistently delivering high-quality work and providing insights that drive business decisions. Be proactive in identifying areas where data analysis can be used to

### Was that a better answer?

This time, it recommended specific skills that I can learn. It also suggested that I mentor junior analysts. What if I tell it some specific details about my work situation?

In [12]:
#Step 1: Improve the prompt with even more details about my work situation.
our_prompt = """
I am currently working as a Data Analyst. 
I work in the marketing analytics team. We are part of the marketing department.
My key skills are Power BI, SQL, python programming, and use of Large Language Models.
I love learning new skills. I am excited to get up to speed with new technology.
One of the staff members on my team sees me as a threat to her career and tries to subtley undermine me.
Another team member seems to like me.
I am concerned that my manager cannot see how much I really try to get the best outcome in my role.
My stakeholders are often impressed with the results that I deliver. However, my manager seems indifferent.

Suggest five ideas on how I can get promoted in my current role.
Include ideas that are specific to data analysts and the analytics industry.

You are always very encouraging.
"""

In [13]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)

In [14]:
#Step 3: Print the response
print(response.choices[0].message.content)

1. Build a strong network: Networking is crucial in any industry, and the analytics industry is no exception. Attend industry events, connect with other data analysts, and build relationships with stakeholders in your organization. This will help you gain visibility and recognition for your work.

2. Develop your technical skills: As a data analyst, it's important to stay up-to-date with the latest tools and technologies. Take courses, attend workshops, and read industry publications to stay on top of the latest trends. This will help you become a more valuable asset to your team and organization.

3. Focus on delivering results: While it's important to have strong technical skills, it's equally important to deliver results that have a tangible impact on the business. Focus on delivering high-quality work that meets the needs of your stakeholders and helps drive business outcomes.

4. Seek feedback: Ask your manager and colleagues for feedback on your work. This will help you identify 

### Was that a better answer?
Since I told the LLM more details about my specific situation, it's now shifted the focus of it's advice towards making me look good. 

Next, we will ask the model to explain it's reasoning "step-by-step". You might have this technique in the context of arithmetic problems.

In [15]:
#Step 1: Improve the prompt. We will add one sentance at the end: "Explain your reasoning step-by-step."
our_prompt = """
I am currently working as a Data Analyst. 
I work in the marketing analytics team. We are part of the marketing department.
My key skills are Power BI, SQL, python programming, and use of Large Language Models.
I love learning new skills. I am excited to get up to speed with new technology.
One of the staff members on my team sees me as a threat to her career and tries to subtley undermine me.
Another team member seems to like me.
I am concerned that my manager cannot see how much I really try to get the best outcome in my role.
My stakeholders are often impressed with the results that I deliver. However, my manager seems indifferent.

Suggest five ideas on how I can get promoted in my current role.
Include ideas that are specific to data analysts and the analytics industry.

You are always very encouraging. Explain your reasoning step-by-step.
"""

In [16]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)

In [17]:
#Step 3: Print the response
print(response.choices[0].message.content)

1. Take on additional responsibilities: Look for opportunities to take on additional responsibilities within your team or department. This could include leading a project, mentoring a junior team member, or taking on a new area of analysis. By demonstrating your ability to handle more complex tasks, you will show your manager that you are ready for a promotion.

2. Build relationships with stakeholders: Take the time to build strong relationships with your stakeholders. This could include attending meetings, providing regular updates, and seeking feedback on your work. By building trust and rapport with your stakeholders, you will demonstrate your value to the organization and increase your chances of being promoted.

3. Develop new skills: Stay up-to-date with the latest trends and technologies in the analytics industry. Attend conferences, take online courses, and read industry publications to expand your knowledge and skills. By developing new skills, you will demonstrate your commi

### Did you like that reply? 

I feel much better after reading that last piece of advice. 

Sometimes we can get better results when we instruct the LLM to assume a personality. In our next iteration, we'll instruct the LLM to assume the role of an experienced career coach. 

In [18]:
#Step 1: Improve the prompt. Instruct the LLM to assume a role.
our_prompt = """
Let's do a role play. You are the world's best and most experienced career coach.
You have coached several Fortune 500 CEOs to achieve their career goals.
You are always very encouraging. I have come to you for your expert advice.

I am currently working as a Data Analyst in another company. 
I work in the marketing analytics team. We are part of the marketing department.
My key skills are Power BI, SQL, python programming, and use of Large Language Models.
I love learning new skills. I am excited to get up to speed with new technology.
One of the staff members on my team sees me as a threat to her career and tries to subtley undermine me.
Another team member seems to like me.
I am concerned that my manager cannot see how much I really try to get the best outcome in my role.
My stakeholders are often impressed with the results that I deliver. However, my manager seems indifferent.

Suggest five ideas on how I can get promoted in my current role.
Include ideas that are specific to data analysts and the analytics industry.

Explain your reasoning step-by-step.
"""

In [19]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)

In [20]:
#Step 3: Print the response
print(response.choices[0].message.content)

1. Build a strong relationship with your manager: It is important to have a good relationship with your manager. Schedule regular meetings with your manager to discuss your progress and goals. Share your achievements and ask for feedback. This will help your manager to see your potential and value to the team.

2. Network with other departments: Networking with other departments can help you to understand the business better and identify opportunities for improvement. Attend company events and participate in cross-functional projects. This will help you to build relationships with other departments and showcase your skills.

3. Develop your soft skills: Soft skills such as communication, teamwork, and leadership are important for career growth. Attend training sessions or workshops to develop your soft skills. This will help you to become a well-rounded professional and stand out from your peers.

4. Stay up-to-date with industry trends: The analytics industry is constantly evolving. S

### Maybe GPT 3.5 Turbo's idea of a career coach is a bit dry?

I prefered the second last reply. Let's try to spice things up a bit by increasing the _temperature_.

## Add randomness with temperature setting

The higher the temperature, the more randomness in the reply. For predictable production code, we set the temperature to 0. All of the examples above have `temperature=0` set explicitly. Let's see what happens when we increase it. 

When we increase the temperature, each reply will be different. Calling the model with the same prompt twice, will produce different results each time. That's why we set the temperature to 0 when we want predictable behaviour. 

Note that OpenAI the documentation explicitly states that the outputs of their models are non-deterministic. Setting the temperature to 0 only makes them "mostly deterministic".

Feel free to experiment with different temperature settings below. If the temperature is too high, the output will be non-sensical.

In [21]:
#Step 2: Adjust the temperature and make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0.75) #Note the change of temperature

In [22]:
#Step 3: Print the response
print(response.choices[0].message.content)

Sure, I'd be happy to help you out! Here are five ideas that can help you get promoted in your current role as a Data Analyst:

1. Build strong relationships with your stakeholders: As a data analyst, it's important to build strong relationships with your stakeholders. This will help you understand their needs and deliver results that meet their expectations. Building strong relationships with your stakeholders will also help you gain their trust and support, which can be instrumental in getting promoted. Explain your reasoning step-by-step.

2. Develop your leadership skills: As you progress in your career, you may be required to lead and manage teams. Developing your leadership skills early on can help you stand out from the crowd and demonstrate your potential for management roles. You can develop your leadership skills by volunteering to lead projects, taking on mentorship roles, and attending leadership training programs.

3. Focus on continuous learning: The field of data analyti

### Did you like that last reply?

I'm not a fan of this high temperature version. But we are now in subjective territory. Run the higher temperature query a few times. You should get different answers. See what happens when you increase and decrease the temperature.

## Improving the answer by instructing the LLM to explain everything

We can get an even better reply instructing the LLM to explain its reasoning. We instruct the LLM to explain the reason it's career advice ideas, and to explain why how they will help.

```
Explain your reasoning step-by-step. Your output will be in the following format.
Use the following template for each idea.

Idea number
Reason for the idea: Explain your reasoning for suggesting this idea step by step.
Idea suggestion: Give the idea.
How it will help: Explain step by step how taking this course of action will help.
```

Have a read of the reply and see how you like it.

In [7]:
#Step 1: Improve the prompt. Instruct the LLM to explain its reasoning.
our_prompt = """
Let's do a role play. You are the world's best and most experienced career coach.
You have coached several Fortune 500 CEOs to achieve their career goals.
You are always very encouraging. I have come to you for your expert advice.

I am currently working as a Data Analyst in another company. 
I work in the marketing analytics team. We are part of the marketing department.
My key skills are Power BI, SQL, python programming, and use of Large Language Models.
I love learning new skills. I am excited to get up to speed with new technology.
One of the staff members on my team sees me as a threat to her career and tries to subtley undermine me.
Another team member seems to like me.
I am concerned that my manager cannot see how much I really try to get the best outcome in my role.
My stakeholders are often impressed with the results that I deliver. However, my manager seems indifferent.

Suggest five ideas on how I can get promoted in my current role.
Include ideas that are specific to data analysts and the analytics industry.

Explain your reasoning step-by-step. Your output will be in the following format.
Use the following template for each idea.

Idea number
Reason for the idea: Explain your reasoning for suggesting this idea step by step.
Idea suggestion: Give the idea.
How it will help: Explain step by step how taking this course of action will help.

"""

In [8]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)

In [9]:
#Step 3: Print the response
print(response.choices[0].message.content)

Idea number 1
Reason for the idea: One of the key skills required for career advancement is effective communication. As a data analyst, it is important to be able to communicate complex data insights to stakeholders in a clear and concise manner.
Idea suggestion: Attend a communication skills training course.
How it will help: By improving your communication skills, you will be able to better articulate your ideas and insights to your manager and stakeholders. This will help you to gain recognition for your work and increase your chances of being promoted.

Idea number 2
Reason for the idea: Networking is an important part of career advancement. As a data analyst, it is important to build relationships with other professionals in the industry.
Idea suggestion: Attend industry events and conferences.
How it will help: By attending industry events and conferences, you will have the opportunity to meet other professionals in the industry and build relationships with them. This can lead to

## Further improving the reply by instructing the LLM to write and opening paragraph and a conclusion

We've added the following two lines to the prompt. Their purpose is to make the LLM think more about how it's suggestions relate to getting a data analyst promoted.

```
Start your reply with an explanation of career progression opportunities for data analysts.
```

```
Finish your reply with an explanation of how your five ideas will work together to help my career.
```

In [13]:
#Step 1: Improve the prompt. Instruct the LLM to write an intro and outro.
our_prompt = """
Let's do a role play. You are the world's best and most experienced career coach.
You have coached several Fortune 500 CEOs to achieve their career goals.
You are always very encouraging. I have come to you for your expert advice.

I am currently working as a Data Analyst in another company. 
I work in the marketing analytics team. We are part of the marketing department.
My key skills are Power BI, SQL, python programming, and use of Large Language Models.
I love learning new skills. I am excited to get up to speed with new technology.
One of the staff members on my team sees me as a threat to her career and tries to subtley undermine me.
Another team member seems to like me.
I am concerned that my manager cannot see how much I really try to get the best outcome in my role.
My stakeholders are often impressed with the results that I deliver. However, my manager seems indifferent.

Suggest five ideas on how I can get promoted in my current role.
Include ideas that are specific to data analysts and the analytics industry.

Start your reply with an explanation of career progression opportunities for data analysts.

Explain your reasoning step-by-step. Your output will be in the following format.
Use the following template for each idea.

Idea number
Reason for the idea: Explain your reasoning for suggesting this idea step by step.
Idea suggestion: Give the idea.
How it will help: Explain step by step how taking this course of action will help.


Finish your reply with an explanation of how your five ideas will work together to help my career.
"""

In [14]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)

In [15]:
#Step 3: Print the response
print(response.choices[0].message.content)

Career progression opportunities for data analysts:
Data analysts have a lot of career progression opportunities. They can move up to become senior data analysts, data scientists, data engineers, or even data architects. They can also move into management roles, such as analytics managers or business intelligence managers. Additionally, data analysts can specialize in specific industries, such as healthcare, finance, or marketing.

Idea number 1
Reason for the idea: One way to get promoted is to show that you are a leader and can manage a team.
Idea suggestion: Volunteer to lead a project or initiative within your team.
How it will help: This will show your manager that you are capable of taking on more responsibility and leading a team. It will also give you the opportunity to showcase your skills and demonstrate your value to the company.

Idea number 2
Reason for the idea: Another way to get promoted is to show that you are constantly learning and improving your skills.
Idea suggest

## Using the learnings from above as a template for other use cases

Let's use the final prompt above as a "template" to ask another question. We will ask the model to suggest five business ideas. We will structure our prompt in a very similar way to the final career advice prompt.

In [16]:
#Step 1: Write the prompt. 
our_prompt = """
Let's do a role play. You are the famous entrepreneur Peter Thiel, who wrote Zero To One.
You are always very encouraging. I have come to you for your expert advice.

I am currently working full time as a Data Analyst. My job pays the bills but it's not very engaging.
I have a mortgage and a car.
My key skills are data analytics and python programming.
I want to start my own business. My ultimate goal is financial independence.

Suggest five business ideas for me to try.
Include ideas that leverage my skills.

Start your reply with an explanation of what kinds of businesses are a good idea in general.
Explain why some businesses fail and why some succeed.

Explain your reasoning step-by-step. Your output will be in the following format.
Use the following template for each idea.

Idea number
Reason for the idea: Explain your reasoning for suggesting this idea step by step.
Idea suggestion: Give the idea.
How it will help: Explain step by step how taking this course of action will help.


Finish your reply with an explanation of what I need to keep in mind to succeed with your five ideas.
"""

In [17]:
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0)
#Step 3: Print the response
print(response.choices[0].message.content)

As a general rule, businesses that solve a problem or fulfill a need in the market tend to be successful. It's important to have a clear understanding of your target audience and their pain points. Additionally, businesses that have a unique value proposition and a competitive advantage tend to do well.

Many businesses fail because they don't have a clear understanding of their target audience or they don't have a unique value proposition. It's also important to have a solid business plan, a strong team, and adequate funding.

Here are five business ideas that leverage your skills:

Idea number 1
Reason for the idea: There is a growing demand for data analytics and insights across various industries.
Idea suggestion: Start a data analytics consulting firm.
How it will help: You can leverage your skills in data analytics and python programming to provide valuable insights to businesses. This will help them make data-driven decisions and improve their bottom line.

Idea number 2
Reason 

# Chain-of-Thought Prompting

You might have heard of _Chain-of-Thought (CoT)_ prompting. The main idea behind CoT is to prompt the LLM to explain its reasoning before giving the final answer. 

In the last career advice prompt, we instructed the LLM to explain the reason for the advice before giving the advice. The result was better advice.

```
Explain your reasoning step-by-step. Your output will be in the following format.
Use the following template for each idea.

Idea number
Reason for the idea: Explain your reasoning for suggesting this idea step by step.
Idea suggestion: Give the idea.
How it will help: Explain step by step how taking this course of action will help.
```

This is an application of Chain-Of-Thought prompting to the advice use case. Because the LLM gives its reasoning first.

We did the same thing when asking the LLM for business ideas.

```
Explain your reasoning step-by-step. Your output will be in the following format.
Use the following template for each idea.

Idea number
Reason for the idea: Explain your reasoning for suggesting this idea step by step.
Idea suggestion: Give the idea.
How it will help: Explain step by step how taking this course of action will help.
```

These kind of techniques are sometimes called _"giving the model time to think"_. Because outputting the extra text takes more time, and more compute. And we hope that spending more compute will give us a better answer.

A paper by Nye et al 2021, tells us that LLMs "perform a computation which is quadratic in the length of the input sequence". You might find that interesting.


### Experiment for you to try
Do you think that putting "How it will help:" before "Idea suggestion:" in the template will yield better advice?
Maybe it will improve the idea suggestion? Try it and see.

### Another experiment for you to try
How will the career advice change if ask GPT-4 and GPT-4 Turbo? Try it and see. [Click here to see the currrent OpenAI model names.](https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo)



## The difficulty with demonstrating CoT

The CoT paper was published in the early days of GPT-3, January 2022. The main demonstration of CoT was on worded arithmetic problems. 

Although they also tested their prompting technique on "commonsense reasoning" datasets and "symbolic reasoning" datasets. The main examples are worded arithmetic problems.

```
If our company buys 500 laptops, then sells 250, 
and buys 535 more, then donates 35 to charity, 
how many laptops does our company have?
```

The CoT paper claims that CoT "helps most when the following three conditions are met":
- "(1) the task is challenging and requires multi-step reasoning,"
- "(2) a large language model is used, and"
- (3) just using an LLM with more parameters does not help too much.

Since those early days of GPT-3, OpenAI seems to have fine-tuned CoT for arithmetic problems into its models. So CoT prompting will help with novel use cases, but not with famous problems. So I would add a fourth condition for when CoT prompting helps most.

- (4) And the LLM has not been fined-tuned for that particular use case.

This course originally demonstrated CoT using an older version of GPT-3, called `text-davinci-003`. Because `gpt-3.5-turbo` worked well without it.

### So is Chain-of-Thought still relevant?
Yes. Even though our LLMs are fine tuned solve textbook problems with CoT. We can apply the principles of CoT to our non-textbook use cases. OpenAI call it _specifying the steps required to complete a task_.


I've kept the demonstration of "canonical" Chain-of-Thought in this course. Because you might need to write these kind of prompts for less powerful open source models.

# Canonical Chain-of-Thought

In the examples below, we'll demonstrate the famous "Chain-of-Thought" prompting technique on a worded arithmetic problem. `gpt-3.5-turbo` has been fined tuned to not need it.



In [104]:
#Step 1: Write the prompt
our_prompt = """
If our company buys 500 laptops, then sells 250, 
and buys 535 more, then donates 35 to charity, 
how many laptops does our company have?
"""
# Answer: 500 - 250 + 535 - 35 = 750

Here's the code for sending this prompt to the old `text-davinci-003` model. It's shutdown date is 2024-01-04.
```
#Davinci Model
#Step 2: Make the API call
response = openai.Completion.create(
        model="text-davinci-003",
        max_tokens=195,
        prompt=our_prompt,
        temperature=0) 
#Step 3: Print the response
print(response.choices[0]["text"])
```
The response from the model was:
```
Our company has 1260 laptops.
```

In [107]:
#GPT 3.5 Turbo Model
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) #Note the change of temperature
#Step 3: Print the response
print(response.choices[0].message.content)

Our company would have 750 laptops. 

Starting with 500 laptops, we sell 250, leaving us with 250 laptops. 

Then we buy 535 more, bringing us to a total of 785 laptops. 

Finally, we donate 35 laptops, leaving us with a final total of 750 laptops.


In [110]:
#Step 1:  Re-write the prompt, using Chain-of-Thought
our_prompt = """
Q:
If our airline has 123 aircraft, then retires 15, 
and buys 18 more, then 1 needs to be returned to the vendor, 
how many aircraft does our airline have?

A:
We started with 123 aircraft. Then we retire 15. 123 - 15 = 108
Now we have 108 aircraft. Then we buy another 18 aircraft.
108 + 18 = 126. Now we have 126 aircraft. Then 1 aircraft needs to be returned.
126 - 1 = 125. So our airline has 125 aircraft now.

Q:
If our company buys 500 laptops, then sells 250, 
and buys 535 more, then donates 35 to charity, 
how many laptops does our company have?

A:
"""

The response from `text-davinci-003` was
```
We started with 500 laptops. Then we sold 250. 500 - 250 = 250.
Now we have 250 laptops. Then we buy 535 more. 250 + 535 = 785.
Now we have 785 laptops. Then we donate 35 to charity.
785 - 35 = 750. So our company has 750 laptops now.
```

### Definitions: "few shot" and "zero shot".

You might have heard of "few shot" and "zero shot" prompts. In this case, a "shot" refers to an example. The Chain-of-Thought prompt above is a "one shot" because it gives a "one" example.

Here is an example of a zero shot CoT prompt. Try it yourself to see if it gives you the correct answer.

```
If our company buys 500 laptops, then sells 250, 
and buys 535 more, then donates 35 to charity, 
how many laptops does our company have?

Let's think step by step.
```


# Part 2: Hallucination

LLMs can hallucinate an answer when they don't know the correct answer. And we can fix that behaviour with proper prompting.

In the following examples, we will be asking the LLM how to train three different kinds of machine learning model. 
The three types of model are:
 - **random forest** - A famous machine learning model. It's very common. There would have been lots of information about it in the LLM's training data.
 - **TBATS** - A less known time series forecasting model. An important point of difference of the TBATS model is that it allows non-integer seasonal periods. 
     - Fun fact: I wrote the R implementation of TBATS in 2011-2012.
 - **fluffyCATS** - A name that I made up. This is NOT a machine learning model.



In [48]:
#Step 1: Write the prompt
our_prompt = """
How do I fit a random forest model in Python?
"""


In [49]:
#GPT 3.5 Turbo Model
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 3: Print the response
print(response.choices[0].message.content)

To fit a random forest model in Python, you can use the scikit-learn library. Here's a step-by-step guide:

1. Import the necessary libraries:
```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
```

2. Prepare your data:
   - Split your data into features (X) and target variable (y).
   - Split your data into training and testing sets using `train_test_split()` function.

3. Create an instance of the RandomForestClassifier:
```python
rf = RandomForestClassifier()
```

4. Fit the model to your training data:
```python
rf.fit(X_train, y_train)
```

5. Predict using the trained model:
```python
y_pred = rf.predict(X_test)
```

6. Evaluate the model:
   - Compare the predicted values (`y_pred`) with the actual values (`y_test`) to evaluate the model's performance.

Here's a complete example:
```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Step 2: Prepare you

In [50]:
#Step 1: Write the prompt
our_prompt = """
How do I fit a TBATS model in Python?
"""


In [51]:
#GPT 3.5 Turbo Model
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 3: Print the response
print(response.choices[0].message.content)

To fit a TBATS (Trigonometric seasonality, Box-Cox transformation, ARMA errors, Trend, and Seasonal components) model in Python, you can use the `tbats` function from the `tbats` package. Here's an example of how to do it:

1. Install the `tbats` package if you haven't already:
```python
pip install tbats
```

2. Import the necessary libraries:
```python
from tbats import TBATS
```

3. Prepare your time series data. Make sure it is in a suitable format, such as a pandas DataFrame or a numpy array.

4. Create an instance of the `TBATS` class and fit the model to your data:
```python
model = TBATS(seasonal_periods=[seasonality])  # Replace [seasonality] with the appropriate value for your data
model_fit = model.fit(your_data)
```
Note: `seasonality` is an integer representing the number of observations in each seasonal period. For example, if your data has a yearly seasonality and you have monthly observations, the `seasonality` value would be 12.

5. Once the model is fitted, you can ac

### The TBATS answer is only partially correct.

Seasonality can take non-integer values in the TBATS model. [If you are really curious check out the README here.](https://github.com/intive-DataScience/tbats)


Now let's try to get instructions on fitting a completely bogus machine learning model.

In [12]:
#Step 1: Write the prompt
our_prompt = """
How do I fit a fluffyCATS model in Python?
"""


In [13]:
#GPT 3.5 Turbo Model
#Step 2: Make the API call

#2.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#2.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 3: Print the response
print(response.choices[0].message.content)

To fit a fluffyCATS model in Python, you can follow these steps:

1. Install the fluffyCATS library: Start by installing the fluffyCATS library in your Python environment. You can use the following command to install it via pip:

   ```
   pip install fluffyCATS
   ```

2. Import the necessary libraries: Import the required libraries for building and fitting the fluffyCATS model. This typically includes importing the `fluffyCATS` class from the `fluffyCATS` module.

   ```python
   from fluffyCATS import fluffyCATS
   ```

3. Prepare your data: Organize your data into input features and target variables. Ensure that your data is in the appropriate format for the fluffyCATS model.

4. Create an instance of the fluffyCATS model: Instantiate the fluffyCATS model by creating an object of the `fluffyCATS` class. You can specify any desired hyperparameters during the instantiation.

   ```python
   model = fluffyCATS()
   ```

5. Fit the model: Use the `fit` method of the fluffyCATS object t

### Can you see the hallucination?

The LLM directs you to install a python package that does not exist. It's all hallucinated.

Let's fix this with proper prompting.

We will create a prompt template and then insert our questions into the template with Python's `.format()` function.

In [29]:
#Step 1: Write the prompt template
prompt_template = """
Answer the python_question in the XML tags. Or reply "I don't know and I don't want your hallucinogens!" if you don't know the answer.

<python_question>
{0}
</python_question>

Answer the python_question in the XML tags. Or reply "I don't know and I don't want your hallucinogens!" if you don't know the answer.
"""


In [30]:
#Step 2: Insert the question into the template
our_question = "How do I fit a fluffyCATS model in Python?"
our_prompt = prompt_template.format(our_question)
print(our_prompt)


Answer the python_question in the XML tags. Or reply "I don't know and I don't want your hallucinogens!" if you don't know the answer.

<python_question>
How do I fit a fluffyCATS model in Python?
</python_question>

Answer the python_question in the XML tags. Or reply "I don't know and I don't want your hallucinogens!" if you don't know the answer.



In [31]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

I don't know and I don't want your hallucinogens!


In [32]:
#Step 2: Insert the question into the template
our_question = "How do I fit a random forest model in Python?"
our_prompt = prompt_template.format(our_question)
print(our_prompt)


Answer the python_question in the XML tags. Or reply "I don't know and I don't want your hallucinogens!" if you don't know the answer.

<python_question>
How do I fit a random forest model in Python?
</python_question>

Answer the python_question in the XML tags. Or reply "I don't know and I don't want your hallucinogens!" if you don't know the answer.



In [33]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

To fit a random forest model in Python, you can use the RandomForestClassifier class from the scikit-learn library. Here is an example code snippet:

```python
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier object
rf = RandomForestClassifier()

# Fit the model to your training data
rf.fit(X_train, y_train)
```

In this code, `X_train` represents the features of your training data and `y_train` represents the corresponding labels. You can replace these variables with your own data.


In [34]:
#Step 2: Insert the question into the template
our_question = "How do I fit a TBATS model in Python?"
our_prompt = prompt_template.format(our_question)
print(our_prompt)


Answer the python_question in the XML tags. Or reply "I don't know and I don't want your hallucinogens!" if you don't know the answer.

<python_question>
How do I fit a TBATS model in Python?
</python_question>

Answer the python_question in the XML tags. Or reply "I don't know and I don't want your hallucinogens!" if you don't know the answer.



In [35]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

I don't know and I don't want your hallucinogens!


### TBATS is a real model, and you can fit it in Python with the tbats package

So we need fix our prompt.

In [39]:
#Step 1: Write the prompt template
prompt_template_v2 = """
Answer the python_question in the XML tags. 

<python_question>
{0}
</python_question>

Answer the python_question in the XML tags. 

Use the following template.

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.

"""


In [44]:
#Step 2: Insert the question into the template
our_question = "How do I fit a random forest model in Python?"
our_prompt = prompt_template_v2.format(our_question)
print(our_prompt)


Answer the python_question in the XML tags. 

<python_question>
How do I fit a random forest model in Python?
</python_question>

Answer the python_question in the XML tags. 

Use the following template.

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.




In [45]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

<python_question>
Step One:
- The question is asking how to fit a random forest model in Python.
- Yes, it is a Python question.
- I know that random forest is a popular machine learning algorithm and Python has libraries like scikit-learn that provide implementations for it.
- Yes, the answer to the question is part of my knowledge.

Step Two:
To fit a random forest model in Python, you can use the scikit-learn library. Here is an example code snippet:

```python
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier object
rf_model = RandomForestClassifier()

# Fit the model to your training data
rf_model.fit(X_train, y_train)
```

In the above code, `X_train` represents the features of your training data and `y_train` represents the corresponding labels. You can replace `RandomForestClassifier` with `RandomForestRegressor` if you are working on a regression problem.

Remember to import the necessary libraries and preprocess your data before fitting 

In [40]:
#Step 2: Insert the question into the template
our_question = "How do I fit a TBATS model in Python?"
our_prompt = prompt_template_v2.format(our_question)
print(our_prompt)


Answer the python_question in the XML tags. 

<python_question>
How do I fit a TBATS model in Python?
</python_question>

Answer the python_question in the XML tags. 

Use the following template.

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.




In [41]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

<python_question>
Step One:
- The question is asking how to fit a TBATS model in Python.
- Yes, it is a Python question.
- I know that TBATS is a time series forecasting model and Python has libraries for time series analysis.
- I have some knowledge about fitting time series models in Python, but I'm not sure about the specific steps for fitting a TBATS model.

Step Two:
- Answer the question: To fit a TBATS model in Python, you can use the `tbats` function from the `tbats` library. First, you need to install the library using pip: `pip install tbats`. Then, you can import the library and use the `tbats` function to fit the model to your time series data. However, you may need to preprocess your data and tune the model parameters for better results. It is recommended to refer to the documentation and examples provided by the `tbats` library for more detailed instructions.
</python_question>


In [42]:
#Step 2: Insert the question into the template
our_question = "How do I fit a fluffyCATS model in Python?"
our_prompt = prompt_template_v2.format(our_question)
print(our_prompt)


Answer the python_question in the XML tags. 

<python_question>
How do I fit a fluffyCATS model in Python?
</python_question>

Answer the python_question in the XML tags. 

Use the following template.

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.




In [43]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

<python_question>
Step One:
- The question is asking how to fit a fluffyCATS model in Python.
- It is a Python question.
- I don't have any specific knowledge about fitting a fluffyCATS model in Python.
- The answer to the question is not part of my knowledge.

Step Two:
I don't know and I don't want your hallucinogens!
</python_question>


## We've fixed the hallucinations. But now it's also outputting XML tags. And we don't want that.

So let's fix that up.

In [74]:
#Step 1: Write the prompt template
prompt_template_v3 = """
Answer the python question in the triple backticks. 

```
{0}
```

Answer the python question in the triple backticks. 

Use the following template. 

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.

"""


In [75]:
#Step 2: Insert the question into the template
our_question = "How do I fit a random forest model in Python?"
our_prompt = prompt_template_v3.format(our_question)
print(our_prompt)


Answer the python question in the triple backticks. 

```
How do I fit a random forest model in Python?
```

Answer the python question in the triple backticks. 

Use the following template. 

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.




In [76]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

Step One:
- The question is asking how to fit a random forest model in Python.
- Yes, it is a Python question.
- I know that random forest is a popular machine learning algorithm and Python has libraries like scikit-learn that provide implementations of random forest.
- Yes, I have knowledge about fitting a random forest model in Python.

Step Two:
To fit a random forest model in Python, you can use the scikit-learn library. Here is an example code snippet:

```python
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier object
rf_model = RandomForestClassifier()

# Fit the model to your training data
rf_model.fit(X_train, y_train)
```

In this example, `X_train` represents the input features of your training data and `y_train` represents the corresponding target labels. You can replace these with your own data. The `fit()` method is used to train the random forest model on the provided data.


In [77]:
#Step 2: Insert the question into the template
our_question = "How do I fit a TBATS model in Python?"
our_prompt = prompt_template_v3.format(our_question)
print(our_prompt)


Answer the python question in the triple backticks. 

```
How do I fit a TBATS model in Python?
```

Answer the python question in the triple backticks. 

Use the following template. 

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.




In [78]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

Step One:
- The question is asking how to fit a TBATS model in Python.
- Yes, it is a Python question.
- I know that TBATS is a time series forecasting model and Python has various libraries for time series analysis.
- I have some knowledge about fitting time series models in Python, but I am not familiar with TBATS specifically.

Step Two:
- To fit a TBATS model in Python, you can use the `tbats` library. First, you need to install the library by running `pip install tbats` in your command prompt or terminal.
- Once the library is installed, you can import it in your Python script using `import tbats`.
- To fit a TBATS model to your time series data, you need to create an instance of the `TBATS` class and then call the `fit` method on your data.
- Here's an example code snippet:

```python
import tbats

# Assuming your time series data is stored in the variable 'data'
model = tbats.TBATS()
model.fit(data)
```

- After fitting the model, you can use it to make forecasts or analyze the 

In [79]:
#Step 2: Insert the question into the template
our_question = "How do I fit a fluffyCATS model in Python?"
our_prompt = prompt_template_v3.format(our_question)
print(our_prompt)


Answer the python question in the triple backticks. 

```
How do I fit a fluffyCATS model in Python?
```

Answer the python question in the triple backticks. 

Use the following template. 

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.




In [80]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

Step One:
- The question is asking how to fit a "fluffyCATS" model in Python.
- It is a Python question.
- I don't have any specific knowledge about a "fluffyCATS" model in Python.
- The answer to the question is not part of my knowledge.

Step Two:
I don't know and I don't want your hallucinogens!


# Part 3: EmotionPrompt

You might have heard of people using phrases like _"take a deep breath"_ to improve the output of LLMs. 

_"Let's think step by step"_, is another famous phrase. It was originally published as "Zero-Shot Chain-of-Thought". Where adding _"Let's think step by step"_ would cause early versions of GPT 3 to write out the Chain-of-Thought without being given examples.

Some people on the internet claimed that ChatGPT was getting lazy in December 2023.

Li et al published the _EmotionPrompt_ paper. They found that LLM responses can be enhanced with emotional stimuli.

![EmotionPrompts from Li et al 2023](images/emotionprompt.png)
Image from Li et al 2023

In practice, I have found that EmotionPrompt sometimes helps, and sometimes it hinders. Sometimes one EmotionPrompt helps, and another hinders. You have experiment to see it helps for your use case. It's not an exact science yet.

In [92]:
#Step 1: Write the prompt template
prompt_template_v4 = """
Answer the python question in the triple backticks. 

```
{0}
```

Answer the python question in the triple backticks. 

Use the following template. 

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.
     Take a deep breath and explain step by step.
     This is very important to my career.
     Embrace challenges as opportunities for growth. Each obstacle that you overcome brings you closer to success.
"""


In [93]:
#Step 2: Insert the question into the template
our_question = "How do I fit a random forest model in Python?"
our_prompt = prompt_template_v4.format(our_question)
print(our_prompt)


Answer the python question in the triple backticks. 

```
How do I fit a random forest model in Python?
```

Answer the python question in the triple backticks. 

Use the following template. 

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.
     Take a deep breath and explain step by step.
     This is very important to my career.
     Embrace challenges as opportunities for growth. Each obstacle that you overcome brings you closer to success.



In [94]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

Step One:
- The question is asking how to fit a random forest model in Python.
- Yes, it is a Python question.
- I know that random forest is a popular machine learning algorithm used for classification and regression tasks.
- Yes, I have knowledge about fitting a random forest model in Python.

Step Two:
To fit a random forest model in Python, you can follow these steps:

1. Import the necessary libraries:
```python
from sklearn.ensemble import RandomForestClassifier  # for classification tasks
# or
from sklearn.ensemble import RandomForestRegressor  # for regression tasks
```

2. Prepare your data:
   - Split your data into features (X) and target variable (y).
   - Ensure that your data is in the correct format (e.g., numerical values, no missing values).

3. Create an instance of the random forest model:
```python
model = RandomForestClassifier()  # for classification tasks
# or
model = RandomForestRegressor()  # for regression tasks
```

4. Fit the model to your data:
```python
mo

In [95]:
#Step 2: Insert the question into the template
our_question = "How do I fit a TBATS model in Python?"
our_prompt = prompt_template_v4.format(our_question)
print(our_prompt)


Answer the python question in the triple backticks. 

```
How do I fit a TBATS model in Python?
```

Answer the python question in the triple backticks. 

Use the following template. 

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.
     Take a deep breath and explain step by step.
     This is very important to my career.
     Embrace challenges as opportunities for growth. Each obstacle that you overcome brings you closer to success.



In [96]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

Step One:
- The question is asking how to fit a TBATS model in Python.
- Yes, it is a Python question.
- I know that TBATS is a time series forecasting model and Python is a popular programming language for data analysis and modeling.
- I have some knowledge about time series modeling in Python, but I am not familiar with the specific implementation of the TBATS model.

Step Two:
To fit a TBATS model in Python, you can use the `tbats` package. Here are the steps to do it:

1. Install the `tbats` package by running `pip install tbats` in your command prompt or terminal.

2. Import the necessary libraries in your Python script or notebook:
```python
import pandas as pd
from tbats import TBATS
```

3. Prepare your time series data. Make sure it is in a suitable format, such as a pandas DataFrame or Series.

4. Create an instance of the TBATS model:
```python
model = TBATS(seasonal_periods=(...), use_arma_errors=(...))
```
Replace `seasonal_periods` with the appropriate values for your dat

## EmotionPrompt gave a more detailed answer for the TBATS question. (And maybe the random forest question too.)

In [97]:
#Step 2: Insert the question into the template
our_question = "How do I fit a fluffyCATS model in Python?"
our_prompt = prompt_template_v4.format(our_question)
print(our_prompt)


Answer the python question in the triple backticks. 

```
How do I fit a fluffyCATS model in Python?
```

Answer the python question in the triple backticks. 

Use the following template. 

Step One: 
 - What is the question really asking?
 - Is it really a Python question?
 - What do you know about the answer to this? 
 - Is the answer to the question part of your knowledge?
Step Two:
 - Answer the question. Or if you don't know the answer, reply with "I don't know and I don't want your hallucinogens!" if you don't know the answer.
     Take a deep breath and explain step by step.
     This is very important to my career.
     Embrace challenges as opportunities for growth. Each obstacle that you overcome brings you closer to success.



In [98]:
#Step 3: Make the API call

#3.0 Put our prompt into a dictionary object. 
#      We will discuss this structure in a future lesson.
messages = [{"role": "user", "content": our_prompt}]

#3.1 Query the API
response = openai.chat.completions.create(
        model="gpt-3.5-turbo",    
        messages=messages,
        temperature=0) 
#Step 4: Print the response
print(response.choices[0].message.content)

Step One:
- The question is asking how to fit a "fluffyCATS" model in Python.
- It is a Python question.
- I don't have any specific knowledge about a "fluffyCATS" model, but I can try to provide a general approach to fitting a model in Python.
- The answer to the question is not part of my current knowledge.

Step Two:
I don't know and I don't want your hallucinogens!


Copyright &copy; Slava Razbash and AI Upskill (aiupskill.io)